In [2]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import librosa
import wave
import pandas as pd
import soundfile as sf
import librosa.display
from PIL import Image
import os
import tensorflow as tf
import scipy.signal as signal
import cv2

In [25]:
def apply_low_pass_filter(audio, sampling_rate, cutoff_freq):
    nyquist_freq = 0.5 * sampling_rate
    normalized_cutoff_freq = cutoff_freq / nyquist_freq
    b, a = signal.butter(4, normalized_cutoff_freq, btype='low', analog=False)
    denoised_audio = signal.lfilter(b, a, audio)
    return denoised_audio

In [26]:
def downsample_audio(audio,original_sampling_rate,target_sampling_rate):
    resampled_audio = librosa.resample(audio, orig_sr=original_sampling_rate, target_sr=target_sampling_rate)
    return resampled_audio

In [27]:
def split_audio(audio, segment_length):
    num_segments = len(audio) // segment_length
    segments = [audio[i*segment_length:(i+1)*segment_length] for i in range(num_segments)]
    return segments

In [28]:
time_shift_factor = 1
pitch_shift_steps = -0.5

def apply_pitch_shift(audio, target_sampling_rate, pitch_shift):
    audio_float = audio.astype(np.float32)  # Remove the normalization step
    shifted_audio = librosa.effects.pitch_shift(audio_float, sr=target_sampling_rate, n_steps=pitch_shift)
    return shifted_audio

def apply_time_shift(audio, time_shift):
    shifted_audio = np.roll(audio, int(time_shift))  # Convert the time shift to an integer
    return shifted_audio

In [10]:
#Augmented-A

def audioAugment(root_directory, output_folder):
    # Iterate through subdirectories and files in the root directory
    for folder_name, subfolders, filenames in os.walk(root_directory):
        # Iterate through audio files in the current folder

        for filename in filenames:
            # Check if the file has an audio extension (e.g., .wav, .mp3, etc.)
            if filename.lower().endswith(('.wav')):
                # Construct the full path to the audio file
                audio_path = os.path.join(folder_name, filename)
                
                audio, sampling_rate = librosa.load(audio_path, sr=None)

                # Denoising
                cutoff_frequency = 195
                denoised_audio = apply_low_pass_filter(audio, sampling_rate, cutoff_frequency)

                # Downsampling
                target_sampling_rate = sampling_rate // 10
                downsampled_audio = downsample_audio(denoised_audio, sampling_rate, target_sampling_rate)

                # Splitting audio
                segment_length = target_sampling_rate * 3
                segments = split_audio(downsampled_audio, segment_length)
                
                time_shifted_segments = [apply_time_shift(segment, time_shift_factor) for segment in segments]

                for i, segment in enumerate(time_shifted_segments):
                    output_path=os.path.join(output_folder,'time_{}_{}'.format(i,filename))
                    sf.write(output_path, segment, target_sampling_rate)
                    
                pitch_shifted_segments = [apply_pitch_shift(segment, target_sampling_rate, pitch_shift_steps) for segment in segments]
                
                for i, segment in enumerate(pitch_shifted_segments):
                    output_path=os.path.join(output_folder,'pitch_{}_{}'.format(i,filename))
                    sf.write(output_path, segment, target_sampling_rate)

In [11]:
import os
from scipy.io import wavfile
import soundfile as sf

def audioSplit(root_directory, output_folder):
    # Iterate through audio files in the root directory
    for folder_name, subfolders, filenames in os.walk(root_directory):
        # Check if the file has an audio extension (e.g., .wav, .mp3, etc.)
        for filename in filenames:
            
            if filename.lower().endswith(('.wav')):
                # Construct the full path to the audio file
                audio_path = os.path.join(root_directory, filename)

                audio, sampling_rate = librosa.load(audio_path, sr=None)

                # Denoising
                cutoff_frequency = 195
                denoised_audio = apply_low_pass_filter(audio, sampling_rate, cutoff_frequency)

                # Downsampling
                target_sampling_rate = sampling_rate // 10
                downsampled_audio = downsample_audio(denoised_audio, sampling_rate, target_sampling_rate)

                # Splitting audio
                segment_length = target_sampling_rate * 3
                segments = split_audio(downsampled_audio, segment_length)
        
                # Write the segments to the output folder
                for i, segment in enumerate(segments):
                    segment_filename = f"segment_{i}_{filename}"
                    segment_path = os.path.join(output_folder, segment_filename)
                    sf.write(segment_path, segment, target_sampling_rate)

In [16]:
#Augmented-A & Split-A/Atraining_artifact
root_directory='/Heart/Dataset_A/Atraining_artifact'
output_folder='/Heart/Dataset_A/Augmented-A/Atraining_artifact'
output_folder1='/Heart/Dataset_A/Split-A/Atraining_artifact'
audioAugment(root_directory, output_folder)
audioSplit(root_directory, output_folder1)

In [17]:
#Augmented-A & Split-A/Atraining_extrahls
root_directory='/Heart/Dataset_A/Atraining_extrahls'
output_folder='/Heart/Dataset_A/Augmented-A/Atraining_extrahls'
output_folder1='/Heart/Dataset_A/Split-A/Atraining_extrahls'
audioAugment(root_directory, output_folder)
audioSplit(root_directory, output_folder1)

In [18]:
#Augmented-A & Split-A/Atraining_murmur
root_directory='/Heart/Dataset_A/Atraining_murmur'
output_folder='/Heart/Dataset_A/Augmented-A/Atraining_murmur'
output_folder1='/Heart/Dataset_A/Split-A/Atraining_murmur'
audioAugment(root_directory, output_folder)
audioSplit(root_directory, output_folder1)

In [19]:
#Augmented-A & Split-A/Atraining_normal
root_directory='/Heart/Dataset_A/Atraining_normal'
output_folder='/Heart/Dataset_A/Augmented-A/Atraining_normal'
output_folder1='/Heart/Dataset_A/Split-A/Atraining_normal'
audioAugment(root_directory, output_folder)
audioSplit(root_directory, output_folder1)

In [59]:
def time_warp(spec, W=5):
    num_bins, num_frames = spec.shape
    spec_aug = np.copy(spec)

    # Generate random control points for time warping
    control_points = [(0, 0), (num_bins-1, num_frames-1)]
    for _ in range(W):
        random_point = np.random.randint(1, num_bins-1)
        control_points.append((random_point, np.random.randint(0, num_frames-1)))

    # Apply time warping
    for i in range(len(control_points)-1):
        c1, c2 = control_points[i], control_points[i+1]
        spec_aug[c1[0]:c2[0]+1, :] = np.roll(spec[c1[0]:c2[0]+1, :], c2[1] - c1[1], axis=1)

    return spec_aug

def frequency_mask(spec, F=30, num_masks=2): #for dataset-A, num_masks=2 ; dataset-B, num_masks=1
    spec_aug = np.copy(spec)

    # Apply frequency masking
    num_bins, num_frames = spec.shape
    for _ in range(num_masks):
        f = np.random.randint(0, F)
        f0 = np.random.randint(0, num_bins - f)
        spec_aug[f0:f0+f, :] = 0

    return spec_aug

In [60]:
#SpecAugment-A
def SpecAugment(root_directory, output_folder):
    # Iterate through subdirectories and files in the root directory
    for folder_name, subfolders, filenames in os.walk(root_directory):
        # Iterate through audio files in the current folder

        for filename in filenames:
            if filename.lower().endswith(('.wav')):
#                 Construct the full path to the audio file
                audio_path = os.path.join(folder_name, filename)
                # Load audio file
                audio, sr = librosa.load(audio_path, sr=None)
                
                target_sampling_rate=sr//10
                # Compute spectrogram
                spectrogram = librosa.feature.melspectrogram(y=audio, sr=target_sampling_rate, n_fft=2048, hop_length=512, n_mels=128)
                spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)
                
                # Normalize the spectrogram to values between 0 and 1
                spectrogram_normalized = (spectrogram_db - np.min(spectrogram_db)) / (np.max(spectrogram_db) - np.min(spectrogram_db))

                # Scale the spectrogram to values between 1 and 255
                spectrogram_scaled = spectrogram_normalized * 255
                
                # Apply time warping
                time_warped_spectrogram = time_warp(spectrogram_scaled)

                # Generate frequency-masked spectrograms
                frequency_masked_spectrogram_1 = frequency_mask(spectrogram_scaled)
                frequency_masked_spectrogram_2 = frequency_mask(spectrogram_scaled)
                
                fig, ax = plt.subplots(figsize=(1.28,1.28))
                
                img1=librosa.display.specshow(time_warped_spectrogram, sr=sr)
                plt.savefig(os.path.join(output_folder,'time_{}.png'.format(filename[:-4])),transparent=True)
                
                img2=librosa.display.specshow(frequency_masked_spectrogram_1, sr=sr)
                plt.savefig(os.path.join(output_folder,'freq_{}.png'.format(filename[:-4])),transparent=True)
                
                img3=librosa.display.specshow(frequency_masked_spectrogram_2, sr=sr)
                plt.savefig(os.path.join(output_folder,'freq2_{}.png'.format(filename[:-4])),transparent=True)
                
                img4=librosa.display.specshow(spectrogram_scaled, sr=sr)
                plt.savefig(os.path.join(output_folder,'spec_{}.png'.format(filename[:-4])),transparent=True)
                
                plt.close()

In [61]:
#SpecAugment/Atraining_artifact

root_directory='/Heart/Dataset_A/Split-A/Atraining_artifact'
output_folder='/Heart/Dataset_A/SpecAugment-A/Atraining_artifact'
SpecAugment(root_directory, output_folder)

In [62]:
#SpecAugment/Atraining_extrahls

root_directory='/Heart/Dataset_A/Split-A/Atraining_extrahls'
output_folder='/Heart/Dataset_A/SpecAugment-A/Atraining_extrahls'
SpecAugment(root_directory, output_folder)

In [63]:
#SpecAugment/Atraining_murmur

root_directory='/Heart/Dataset_A/Split-A/Atraining_murmur'
output_folder='/Heart/Dataset_A/SpecAugment-A/Atraining_murmur'
SpecAugment(root_directory, output_folder)

In [64]:
#SpecAugment/Atraining_normal

root_directory='/Heart/Dataset_A/Split-A/Atraining_normal'
output_folder='/Heart/Dataset_A/SpecAugment-A/Atraining_normal'
SpecAugment(root_directory, output_folder)

In [65]:
#Spec

def Spec(root_directory, output_folder):
    # Iterate through subdirectories and files in the root directory
    for folder_name, subfolders, filenames in os.walk(root_directory):
        # Iterate through audio files in the current folder

        for filename in filenames:
            if filename.lower().endswith(('.wav')):
#                 Construct the full path to the audio file
                audio_path = os.path.join(folder_name, filename)
                # Load audio file
                audio, sr = librosa.load(audio_path, sr=None)
                
                target_sampling_rate=sr//10
                # Compute spectrogram
                spectrogram = librosa.feature.melspectrogram(y=audio, sr=target_sampling_rate, n_fft=2048, hop_length=512, n_mels=128)
                spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)
#                 spectrogram_db= cv2.resize(spectrogram_db,(128,128))
                # Normalize the spectrogram to values between 0 and 1
#                 spectrogram_normalized = (spectrogram_db - np.min(spectrogram_db)) / (np.max(spectrogram_db) - np.min(spectrogram_db))

#                 # Scale the spectrogram to values between 1 and 255
#                 spectrogram_scaled = spectrogram_normalized * 255
                
                fig, ax = plt.subplots(figsize=(1.28,1.28))
                
                img1=librosa.display.specshow(spectrogram_db, sr=sr)
                plt.savefig(os.path.join(output_folder,'spec_{}.png'.format(filename[:-4])),transparent=True)
                
                plt.close()

In [66]:
#Spec/Atraining_artifact

root_directory='/Heart/Dataset_A/Augmented-A/Atraining_artifact'
output_folder='/Heart/Dataset_A/Spec-A/Atraining_artifact'
Spec(root_directory, output_folder)

In [67]:
#Spec/Atraining_extrahls

root_directory='/Heart/Dataset_A/Augmented-A/Atraining_extrahls'
output_folder='/Heart/Dataset_A/Spec-A/Atraining_extrahls'
Spec(root_directory, output_folder)

In [68]:
#Spec/Atraining_murmur

root_directory='/Heart/Dataset_A/Augmented-A/Atraining_murmur'
output_folder='/Heart/Dataset_A/Spec-A/Atraining_murmur'
Spec(root_directory, output_folder)

In [69]:
#Spec/Atraining_normal

root_directory='/Heart/Dataset_A/Augmented-A/Atraining_normal'
output_folder='/Heart/Dataset_A/Spec-A/Atraining_normal'
Spec(root_directory, output_folder)

In [70]:
from PIL import Image
spec_data=[]

In [71]:
def data(root_directory,final_class_labels):
    for folder_name, subfolders, filenames in os.walk(root_directory):
        # Iterate through audio files in the current folder
        for filename in filenames:
            if filename.lower().endswith(('.png')):
#                 Construct the full path to the audio file
                audio_path = os.path.join(folder_name, filename)
                img=Image.open(audio_path).convert('RGB')
                img=img.resize((128,128))
                img_array = np.asarray(img)
                img_array = img_array / 255
                spec_data.append([img_array,final_class_labels])   

In [72]:
data('/Heart/Dataset_A/Spec-A/Atraining_artifact','artifact')
data('/Heart/Dataset_A/SpecAugment-A/Atraining_artifact','artifact')

In [73]:
data('/Heart/Dataset_A/Spec-A/Atraining_extrahls','extrahls')
data('/Heart/Dataset_A/SpecAugment-A/Atraining_extrahls','extrahls')

In [74]:
data('/Heart/Dataset_A/Spec-A/Atraining_murmur','murmur')
data('/Heart/Dataset_A/SpecAugment-A/Atraining_murmur','murmur')

In [75]:
data('/Heart/Dataset_A/Spec-A/Atraining_normal','normal')
data('/Heart/Dataset_A/SpecAugment-A/Atraining_normal','normal')

In [76]:
import pandas as pd
spec_data=pd.DataFrame(spec_data,columns=['feature','class'])
spec_data.head()

,feature,class
0,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",artifact
1,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",artifact
2,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",artifact
3,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",artifact
4,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",artifact


In [77]:
spec_data['class'].value_counts()

class
artifact    720
normal      408
murmur      396
extrahls    222
Name: count, dtype: int64

In [78]:
X=np.array(spec_data['feature'].tolist())
y=np.array(spec_data['class'].tolist())

In [79]:
X.shape

(1746, 128, 128, 3)

In [80]:
#downsampling artifact in dataset-A (drop 50%)

import numpy as np

# Identify the class label you want to downsample
class_label = 'artifact'  # Update with your desired class label

# Separate the data into eliminated and remaining subsets
eliminated_subset_X = X[y == class_label]
eliminated_subset_y = y[y == class_label]
remaining_subset_X = X[y != class_label]
remaining_subset_y = y[y != class_label]

# Randomly select 50% of the data from the remaining subset
num_eliminated_samples = len(eliminated_subset_X)
num_samples_to_select = int(num_eliminated_samples * 0.5)

random_indices = np.random.choice(num_eliminated_samples, size=num_samples_to_select, replace=False)
randomly_selected_X = eliminated_subset_X[random_indices]
randomly_selected_y = eliminated_subset_y[random_indices]

# Combine the eliminated subset and the randomly selected subset
downsampled_X = np.concatenate((remaining_subset_X, randomly_selected_X), axis=0)
downsampled_y = np.concatenate((remaining_subset_y, randomly_selected_y), axis=0)


In [81]:
X=downsampled_X
y=np.array(pd.get_dummies(downsampled_y))

In [82]:
X=np.concatenate((X, X), axis=0)
y=np.concatenate((y, y), axis=0)

In [83]:
X.shape

(2772, 128, 128, 3)

In [84]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [85]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

In [51]:
# X_train=np.concatenate((X_train, X_train), axis=0)
# y_train=np.concatenate((y_train, y_train), axis=0)

In [86]:
X_train.shape

(1884, 128, 128, 3)

In [87]:
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout

# Load the pre-trained ResNet152V2 model
resnet152v2 = ResNet152V2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the pre-trained layers
for layer in resnet152v2.layers:
    layer.trainable = False

# Add custom layers on top of the pre-trained model
flatten_layer = Flatten()(resnet152v2.output)
dense_layer = Dense(128, activation='relu')(flatten_layer)
dropout_layer = Dropout(0.35)(dense_layer)
classification_layer = Dense(4, activation='softmax')(dropout_layer)  # Replace 'num_classes' with the number of classes in your problem

# Create the final model
model = Model(inputs=resnet152v2.input, outputs=classification_layer)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_4[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 pool1_pad (ZeroPadding2D)      (None, 66, 66, 64)   0           ['conv1_conv[0][0]']       

 vation)                                                                                          
                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, 32, 32, 64)   16384       ['conv2_block3_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv2_block3_1_bn (BatchNormal  (None, 32, 32, 64)  256         ['conv2_block3_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 32, 32, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
          

 conv3_block2_2_bn (BatchNormal  (None, 16, 16, 128)  512        ['conv3_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block2_2_relu (Activatio  (None, 16, 16, 128)  0          ['conv3_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block2_3_conv (Conv2D)   (None, 16, 16, 512)  66048       ['conv3_block2_2_relu[0][0]']    
                                                                                                  
 conv3_block2_out (Add)         (None, 16, 16, 512)  0           ['conv3_block1_out[0][0]',       
                                                                  'conv3_block2_3_conv[0][0]']    
          

 ization)                                                                                         
                                                                                                  
 conv3_block5_1_relu (Activatio  (None, 16, 16, 128)  0          ['conv3_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block5_2_pad (ZeroPaddin  (None, 18, 18, 128)  0          ['conv3_block5_1_relu[0][0]']    
 g2D)                                                                                             
                                                                                                  
 conv3_block5_2_conv (Conv2D)   (None, 16, 16, 128)  147456      ['conv3_block5_2_pad[0][0]']     
                                                                                                  
 conv3_blo

 conv3_block8_preact_bn (BatchN  (None, 16, 16, 512)  2048       ['conv3_block7_out[0][0]']       
 ormalization)                                                                                    
                                                                                                  
 conv3_block8_preact_relu (Acti  (None, 16, 16, 512)  0          ['conv3_block8_preact_bn[0][0]'] 
 vation)                                                                                          
                                                                                                  
 conv3_block8_1_conv (Conv2D)   (None, 16, 16, 128)  65536       ['conv3_block8_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv3_block8_1_bn (BatchNormal  (None, 16, 16, 128)  512        ['conv3_block8_1_conv[0][0]']    
 ization) 

...

                                                                                                  
 conv4_block29_1_bn (BatchNorma  (None, 8, 8, 256)   1024        ['conv4_block29_1_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv4_block29_1_relu (Activati  (None, 8, 8, 256)   0           ['conv4_block29_1_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv4_block29_2_pad (ZeroPaddi  (None, 10, 10, 256)  0          ['conv4_block29_1_relu[0][0]']   
 ng2D)                                                                                            
                                                                                                  
 conv4_blo

                                                                  'conv4_block31_3_conv[0][0]']   
                                                                                                  
 conv4_block32_preact_bn (Batch  (None, 8, 8, 1024)  4096        ['conv4_block31_out[0][0]']      
 Normalization)                                                                                   
                                                                                                  
 conv4_block32_preact_relu (Act  (None, 8, 8, 1024)  0           ['conv4_block32_preact_bn[0][0]']
 ivation)                                                                                         
                                                                                                  
 conv4_block32_1_conv (Conv2D)  (None, 8, 8, 256)    262144      ['conv4_block32_preact_relu[0][0]
                                                                 ']                               
          

                                                                                                  
 conv4_block34_2_bn (BatchNorma  (None, 8, 8, 256)   1024        ['conv4_block34_2_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv4_block34_2_relu (Activati  (None, 8, 8, 256)   0           ['conv4_block34_2_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv4_block34_3_conv (Conv2D)  (None, 8, 8, 1024)   263168      ['conv4_block34_2_relu[0][0]']   
                                                                                                  
 conv4_block34_out (Add)        (None, 8, 8, 1024)   0           ['conv4_block33_out[0][0]',      
          

 conv5_block1_1_conv (Conv2D)   (None, 4, 4, 512)    524288      ['conv5_block1_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv5_block1_1_bn (BatchNormal  (None, 4, 4, 512)   2048        ['conv5_block1_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block1_1_relu (Activatio  (None, 4, 4, 512)   0           ['conv5_block1_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block1_2_pad (ZeroPaddin  (None, 6, 6, 512)   0           ['conv5_block1_1_relu[0][0]']    
 g2D)     

 n)                                                                                               
                                                                                                  
 conv5_block3_3_conv (Conv2D)   (None, 4, 4, 2048)   1050624     ['conv5_block3_2_relu[0][0]']    
                                                                                                  
 conv5_block3_out (Add)         (None, 4, 4, 2048)   0           ['conv5_block2_out[0][0]',       
                                                                  'conv5_block3_3_conv[0][0]']    
                                                                                                  
 post_bn (BatchNormalization)   (None, 4, 4, 2048)   8192        ['conv5_block3_out[0][0]']       
                                                                                                  
 post_relu (Activation)         (None, 4, 4, 2048)   0           ['post_bn[0][0]']                
          

In [88]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint('heart_resnet_valid.hdf5',save_best_only=True,save_weights_only=False,monitor='val_accuracy',model='max')

In [90]:
#dense-128 and x*2(x_valid)
num_epochs = 20
num_batch_size = 32
model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_valid, y_valid), callbacks=[checkpointer], verbose=1)

Epoch 1/20
59/59 [==============================] - 56s 952ms/step - loss: 0.5045 - accuracy: 0.7951 - val_loss: 0.4030 - val_accuracy: 0.8589
Epoch 2/20
59/59 [==============================] - 39s 661ms/step - loss: 0.3224 - accuracy: 0.8763 - val_loss: 0.2667 - val_accuracy: 0.8889
Epoch 3/20
59/59 [==============================] - 35s 598ms/step - loss: 0.2284 - accuracy: 0.9188 - val_loss: 0.2151 - val_accuracy: 0.9189
Epoch 4/20
59/59 [==============================] - 37s 622ms/step - loss: 0.1864 - accuracy: 0.9321 - val_loss: 0.1627 - val_accuracy: 0.9339
Epoch 5/20
59/59 [==============================] - 38s 638ms/step - loss: 0.1406 - accuracy: 0.9496 - val_loss: 0.1374 - val_accuracy: 0.9369
Epoch 6/20
59/59 [==============================] - 34s 568ms/step - loss: 0.0988 - accuracy: 0.9650 - val_loss: 0.1514 - val_accuracy: 0.9429
Epoch 7/20
59/59 [==============================] - 36s 603ms/step - loss: 0.0775 - accuracy: 0.9713 - val_loss: 0.1457 - val_accuracy: 0.9309

In [91]:
test_accuracy=model.evaluate(X_test,y_test)

18/18 [==============================] - 14s 780ms/step - loss: 0.0524 - accuracy: 0.9820


In [93]:
test_accuracy[1]

0.9819819927215576

In [37]:
# from tensorflow.keras.callbacks import ModelCheckpoint
# checkpointer = ModelCheckpoint('heart_resnet_a.hdf5',save_best_only=True,save_weights_only=False,monitor='val_accuracy',model='max')

In [176]:
# from tensorflow.keras.callbacks import ModelCheckpoint
# checkpointer = ModelCheckpoint('heart_resnet.hdf5',save_best_only=True,save_weights_only=False,monitor='val_accuracy',model='max')

In [38]:
#dense-128 and x*2(without valid data)
num_epochs = 20
num_batch_size = 32
model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)

Epoch 1/20
58/58 [==============================] - 58s 866ms/step - loss: 1.1327 - accuracy: 0.7083 - val_loss: 0.3504 - val_accuracy: 0.8442
Epoch 2/20
58/58 [==============================] - 36s 624ms/step - loss: 0.3075 - accuracy: 0.8782 - val_loss: 0.3359 - val_accuracy: 0.8636
Epoch 3/20
58/58 [==============================] - 35s 601ms/step - loss: 0.2009 - accuracy: 0.9280 - val_loss: 0.2438 - val_accuracy: 0.9026
Epoch 4/20
58/58 [==============================] - 34s 592ms/step - loss: 0.1623 - accuracy: 0.9351 - val_loss: 0.1870 - val_accuracy: 0.9589
Epoch 5/20
58/58 [==============================] - 34s 596ms/step - loss: 0.1227 - accuracy: 0.9627 - val_loss: 0.1793 - val_accuracy: 0.9459
Epoch 6/20
58/58 [==============================] - 34s 591ms/step - loss: 0.0993 - accuracy: 0.9621 - val_loss: 0.1685 - val_accuracy: 0.9632
Epoch 7/20
58/58 [==============================] - 34s 588ms/step - loss: 0.0423 - accuracy: 0.9897 - val_loss: 0.1565 - val_accuracy: 0.9697

In [21]:
#dense-256 and x*2 (without valid data)
num_epochs = 20
num_batch_size = 32
model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)

Epoch 1/20


2023-07-01 11:47:25.483810: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


58/58 [==============================] - 48s 737ms/step - loss: 1.5275 - accuracy: 0.6824 - val_loss: 0.3962 - val_accuracy: 0.7987
Epoch 2/20
58/58 [==============================] - 41s 711ms/step - loss: 0.3334 - accuracy: 0.8615 - val_loss: 0.3393 - val_accuracy: 0.8745
Epoch 3/20
58/58 [==============================] - 35s 604ms/step - loss: 0.2692 - accuracy: 0.8864 - val_loss: 0.2623 - val_accuracy: 0.8896
Epoch 4/20
58/58 [==============================] - 35s 609ms/step - loss: 0.1654 - accuracy: 0.9394 - val_loss: 0.1951 - val_accuracy: 0.9524
Epoch 5/20
58/58 [==============================] - 33s 574ms/step - loss: 0.1245 - accuracy: 0.9605 - val_loss: 0.2145 - val_accuracy: 0.9286
Epoch 6/20
58/58 [==============================] - 36s 622ms/step - loss: 0.1018 - accuracy: 0.9686 - val_loss: 0.1782 - val_accuracy: 0.9545
Epoch 7/20
58/58 [==============================] - 34s 591ms/step - loss: 0.0706 - accuracy: 0.9811 - val_loss: 0.1566 - val_accuracy: 0.9675
Epoch 8/20

In [39]:
y_pred=model.predict(X_test)

15/15 [==============================] - 17s 636ms/step


In [94]:
d = {0: 'Artifact', 1: 'Extrasystole', 2: 'Murmur', 3: 'Normal'}

def pred(audio_path):
    heart_class=[]
    heart_confidence=[]

    audio, sampling_rate = librosa.load(audio_path, sr=None)

    # Denoising
    cutoff_frequency = 195
    denoised_audio = apply_low_pass_filter(audio, sampling_rate, cutoff_frequency)

    # Downsampling
    target_sampling_rate = sampling_rate // 10
    downsampled_audio = downsample_audio(denoised_audio, sampling_rate, target_sampling_rate)

    # Splitting audio
    segment_length = target_sampling_rate * 3
    segments = split_audio(downsampled_audio, segment_length)
    
    for segment in segments:

        spectrogram = librosa.feature.melspectrogram(y=segment, sr=target_sampling_rate)

        # Convert to decibels
        spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)

        # Plot spectrogram
        plt.figure(figsize=(1.28,1.28))
        librosa.display.specshow(spectrogram_db, sr=target_sampling_rate)
        plt.savefig('spectrogram.png',transparent=True)
        plt.close()

        img=Image.open('spectrogram.png').convert('RGB')
        img_arr=np.asarray(img)
        img_arr=img_arr/255

        img_arr = img_arr.reshape(1, 128, 128, 3)
        
        prediction = model.predict(img_arr)
        x=np.argmax(prediction)
        confidence = prediction[0, x]
        heart_class.append(d[x])
        heart_confidence.append(confidence)
        os.remove('spectrogram.png')
    print(heart_class[heart_confidence.index(max(heart_confidence))])

In [95]:
pred('/Downloads/Artifact.wav')

1/1 [==============================] - 0s 102ms/step
Artifact


In [96]:
audio_path='/Heart/Dataset_A/Atraining_artifact/201106121242.wav'
pred(audio_path)

1/1 [==============================] - 0s 78ms/step
Artifact


In [104]:
audio_path='/Heart/Dataset_A/Atraining_murmur/201108222255.wav'
pred(audio_path)

1/1 [==============================] - 0s 87ms/step
Murmur


In [98]:
audio_path='/Heart/Dataset_A/Atraining_normal/201102260502.wav'
pred(audio_path)

1/1 [==============================] - 0s 84ms/step
Normal


In [99]:
audio_path='/Heart/Dataset_A/Atraining_extrahls/201101070953.wav'
pred(audio_path)

1/1 [==============================] - 0s 90ms/step
Extrasystole


In [100]:
audio_path='/Heart/Dataset_A/Atraining_extrahls/201104270458.wav'
pred(audio_path)

1/1 [==============================] - 0s 74ms/step
Extrasystole


In [103]:
audio_path='/Heart/Dataset_A/Atraining_normal/201103140132.wav'
pred(audio_path)

1/1 [==============================] - 0s 114ms/step
Normal
